In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Importação das bibliotecas necessárias
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

In [0]:
def bovw_gerarDicionario(lista_descritores):

    kmeans = KMeans(n_clusters = 256)
    kmeans = kmeans.fit(lista_descritores)
    global dicionario
    dicionario = kmeans.cluster_centers_

In [0]:
def getDescritores(img_caminho):
    ALTURA = 360
    LARGURA = 360

    # Ler imagens
    img_teste = cv2.imread(img_caminho) #RGB
    img_cinza = cv2.cvtColor(img_teste, cv2.COLOR_BGR2GRAY)
    #img_teste = cv2.imread('Img_test1.png',0) #GRAY SCALE

    # Redimensionar
    img_redimencionada = cv2.resize(img_cinza, (LARGURA, ALTURA),interpolation=cv2.INTER_CUBIC)

    # Remover o ruído (suavizar a imagem)
    img_suavizada = cv2.GaussianBlur(img_redimencionada, (7,7),1)
    img_equalizada = cv2.equalizeHist(img_suavizada)

    orb = cv2.ORB_create(nfeatures = 512)

    # Determinar key points
    pontos_chave = orb.detect(img_equalizada, None)

    pontos_chave, descritores = orb.compute(img_equalizada, pontos_chave)

    return descritores

In [0]:
def bovw_computarDescritores(descritores):
    algoritmo_knn = NearestNeighbors(n_neighbors=1)
    algoritmo_knn.fit(dicionario)
    res = algoritmo_knn.kneighbors(descritores, return_distance=False).flatten()
    histograma_caracteristicas = np.histogram(res, bins=np.arange(dicionario.shape[0]))[0]
    return histograma_caracteristicas

In [0]:
def salvarDescritores(descritores, caminho):
    descritores = descritores.reshape((1,descritores.size))

    arquivo=open(os.path.join(caminho, 'orb_descritores.csv'),'a')

    np.savetxt(arquivo, descritores, delimiter=',', fmt='%i')
    arquivo.close()

# MAIN - Geração de características

In [0]:
global dicionario

dados_treinamento = ['drive/My Drive/Aulas_alura/INRIAPerson_Dataset/treinamento/positivos/',
'drive/My Drive/Aulas_alura/INRIAPerson_Dataset/treinamento/negativos/']

# Rotina para criação do dicionário de palavras virtuais
descritores = np.empty((0,32), dtype=np.uint8)
for caminho in dados_treinamento:

    # r=raiz, d=diretorios, a = arquivos
    for r, d, a in os.walk(caminho):
        for arquivo in a: #Somente 400 exemplos positivos e 400 exemplos negativos
            if '.png' in arquivo:
                orb_descritor = getDescritores(os.path.join(r, arquivo))
                descritores = np.append(descritores, orb_descritor, axis=0)

# Bag of Visual words - gerar dicionário de palavras
bovw_gerarDicionario(descritores)

In [0]:
# computar descritores de cada imagem separadamente
for caminho in dados_treinamento:
    # r=raiz, d=diretorios, a = arquivos
    for r, d, a in os.walk(caminho):
        for arquivo in a: #Somente 400 exemplos positivos e 400 exemplos negativos
            if '.png' in arquivo:
                descritor = getDescritores(os.path.join(r, arquivo))
                hitograma_descritor = bovw_computarDescritores(descritor)
                salvarDescritores(hitograma_descritor, caminho)

# Rotina de classificação

In [0]:
def carregarDescritores(caminho):

    descritores = np.loadtxt(os.path.join(caminho, 'orb_descritores.csv'), delimiter=',')
    return descritores

In [0]:
dados_teste = ['drive/My Drive/Aulas_alura/INRIAPerson_Dataset/teste/positivos/',
'drive/My Drive/Aulas_alura/INRIAPerson_Dataset/teste/negativos/']

#Carregar os descritores salvos
descritores = np.empty((0,255))
for caminho in dados_treinamento:
    descritores = np.append(descritores, carregarDescritores(caminho), axis=0)

#KNN para classificar as imagens

rotulos = np.ones(400, dtype=np.uint8) #Rotular as primeiras 400 caracteristicas como 1 - Exemplos sem imagem
rotulos = np.append(rotulos, np.zeros(400)) #Rotular as outras 400 como zero - Exemplos sem imagem

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(descritores, rotulos)

# Ler imagens. 50 positivas e 50 negativas

img_testDescritores = np.empty((0,255), dtype=np.uint8)
for caminho in dados_teste:

    # r=raiz, d=diretorios, a = arquivos
    for r, d, a in os.walk(caminho):
        for arquivo in a:
            if '.png' in arquivo:
                img_descritor = getDescritores(os.path.join(r, arquivo))
                img_descritor = bovw_computarDescritores(img_descritor)
                img_dim_expandida = np.expand_dims(img_descritor, axis=0)
                img_testDescritores = np.append(img_testDescritores, img_dim_expandida, axis=0)

rotulos_teste = np.concatenate((np.ones(100, dtype=np.uint8), np.zeros(100, dtype=np.uint8))) 
print(knn.score(img_testDescritores, rotulos_teste))